In [9]:
from copy import deepcopy
from functools import partial
import bisect 

move2coord = {"Up":(-1,0),
              "Down":(1,0),
              "Right":(0,1),
              "Left":(0,-1)}

pos = lambda mv: move2coord[mv]

goalSt = [[1,2,3],
          [4,5,6],
          [7,8,0]]

def find_num(num, s):
    
    for ir, row in enumerate(s):
        for ic, elem in enumerate(row):
            if elem == num: return ir, ic
            
find_zero = partial(find_num, 0)         

def addTuple(t1, t2):
    return tuple(i+j for i, j in zip(t1, t2))

def is_bound(pos):
    x,y = pos
    if 0<=x<=2 and 0<=y<=2: return True
    return False

def swap(new, actual, s):
    nx, ny = new
    ax, ay = actual
    s[ax][ay], s[nx][ny] = s[nx][ny], s[ax][ay]
    return s

In [56]:
def move(direc, st):
    s, t = st
    zero_pos = find_zero(s)
    new_zero_pos = addTuple(zero_pos, pos(direc))
    
    new_s = deepcopy(s)
    
    if is_bound(new_zero_pos):
        new_s = swap(zero_pos, new_zero_pos, new_s)
    else:
        new_s = "stop"
    return (new_s, t + [direc])
        
def moves(sts):
    return [move(direc, st) for st in sts 
                            for direc in move2coord.keys()
                            if move(direc, st)[0] != "stop"]

def goal(goalSt, si):
    s, t = si
    if s == goalSt: return True
    return False
    
is_goal = partial(goal, goalSt)

def is_feasible(st):
    return True

def absDiff(x, y):
    return (abs(x[0]-y[0]), abs(x[1]-y[1]))
    
def dist(v, x, y):
    x, y = absDiff( find_num(v,x), find_num(v,y) )
    return x+y

sort_by_f = lambda ls: sorted(ls,key=lambda x:x[0])

def f1(st):
    s, t = st
    return len(t) + sum([dist(v, s, goalSt) for v in range(1,9)])

def astar(sts, f):
    if len(sts) == 0:
        print("Couldn't find a feasible solution")
        return ([],[]), 0
        
    goal = list(filter(is_goal, sts))  
    n = 0  
    explore_sts = []
    explore_fs = []
    while len(goal) == 0:
        
        for f_val, st in zip(map(f, sts), sts): # apply f only in the new states
            idx = bisect.bisect(explore_fs, f_val)  # finds the correct position in the val_index
            explore_sts.insert(idx, st)  # insert in the corret position
            explore_fs.insert(idx, f_val)  # insert in the corret position
        
        s = explore_sts.pop(0)
        _ = explore_fs.pop(0)
        sts = moves([s])
        
        goal = list(filter(is_goal, sts))    
        n += len(sts)   
        
    return goal[0], n

def bfs(sts):
    
    if len(sts) == 0:
        print("Could not find a feasible solution")
        return ([],[]),0
    
    goal = list(filter(is_goal, sts))
    n = 0
    
    while len(goal) == 0:
        sts = list(filter(is_feasible, moves(sts)))
        goal = list(filter(is_goal, sts))
        n += len(sts)
    return goal[0], n

In [57]:
s0 = [[1,8,2],
      [0,4,3],
      [7,6,5]]
st = (s0,[])
result,n = bfs([st])
print(n)

18723


In [66]:
s0 = [[1,8,2],
      [0,4,3],
      [7,6,5]]
st = (s0,[])

astar([st], f1)

(([[1, 2, 3], [4, 5, 6], [7, 8, 0]],
  ['Right', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Down']),
 32)

In [55]:
def pretty_print(s):
    for row in s:
        print(row)
    print()

pretty_print(s0)

s = deepcopy(s0)
for direc in result[1]:
    print(direc)
    
    s,i = move(direc,(s,[]))
    print()
    pretty_print(s)

    

[1, 8, 2]
[0, 4, 3]
[7, 6, 5]

Right

[1, 8, 2]
[4, 0, 3]
[7, 6, 5]

Up

[1, 0, 2]
[4, 8, 3]
[7, 6, 5]

Right

[1, 2, 0]
[4, 8, 3]
[7, 6, 5]

Down

[1, 2, 3]
[4, 8, 0]
[7, 6, 5]

Down

[1, 2, 3]
[4, 8, 5]
[7, 6, 0]

Left

[1, 2, 3]
[4, 8, 5]
[7, 0, 6]

Up

[1, 2, 3]
[4, 0, 5]
[7, 8, 6]

Right

[1, 2, 3]
[4, 5, 0]
[7, 8, 6]

Down

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

